<a href='http://www.holoviews.org'><img src="assets/header_logo.png" alt="HoloViews logo" width="20%;" align="left"/></a>
<div style="float:right;"><h2>06. Custom Interactivity</h2></div>

In the [exploring with containers](./03-exploration-with-containers.ipynb) section, the [``DynamicMap``](http://holoviews.org/reference/containers/bokeh/DynamicMap.html) container was introduced. In that section, the arguments to the callable returning elements was supplied by HoloViews sliders. In this section, we will generalize the ways in which you can generate values to update a ``DynamicMap``.

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh', 'matplotlib')
%opts Ellipse [xaxis=None yaxis=None] (color='red' line_width=2)
%opts Box [xaxis=None yaxis=None] (color='blue' line_width=2)

## A simple ``DynamicMap``

Let us now create a simple [``DynamicMap``](http://holoviews.org/reference/containers/bokeh/DynamicMap.html) using three *annotation* elements, namely [``Box``](http://holoviews.org/reference/elements/bokeh/Ellipse.html), [``Text``](http://holoviews.org/reference/elements/bokeh/Ellipse.html) and [``Ellipse``](http://holoviews.org/reference/elements/bokeh/Ellipse.html):

In [ ]:
def annotations(angle):
    radians = (angle / 180) * np.pi
    return (hv.Box(0,0,4, orientation=np.pi/4) 
            * hv.Ellipse(0,0,(2,4), orientation=radians) 
            * hv.Text(0,0,'{0}º'.format(float(angle))))

hv.DynamicMap(annotations, kdims=['angle']).redim.range(angle=(0, 360)).redim.label(angle='angle (º)')

This example uses the concepts introduced  in the [exploring with containers](./03-exploration-with-containers.ipynb) section and as before, the argument ``angle`` is supplied by the position of the 'angle' slider.

## Introducing Streams

HoloViews offers a way of supplying the ``angle`` value to our annotation function through means other than sliders, namely via the *streams* system which you can learn about in our [responding to events](http://holoviews.org/user_guide/Responding_to_Events.html) user guide.
 
 All stream classes are found in the ``streams`` submodule and are subclasses of ``Stream``. You can use ``Stream`` directly to make custom stream classes via the ``define`` classmethod:

In [ ]:
from holoviews.streams import Stream
Angle = Stream.define('Angle', angle=0)

Here ``Angle`` is capitalized as it is a parameterized sub*class* of ``Stream`` with a numeric *angle* parameter, with a default value of zero. You can verify this using ``hv.help``:

In [ ]:
hv.help(Angle)

Now we can declare a ``DynamicMap`` where instead of specifying ``kdims``, we instantiate ``Angle`` with an ``angle`` of 45º and pass it to the ``streams`` parameter of the [``DynamicMap``](http://holoviews.org/reference/containers/bokeh/DynamicMap.html):

In [ ]:
dmap=hv.DynamicMap(annotations, streams=[Angle(angle=45)])
dmap

As expected, we see our ellipse with an angle of 45º as specified via the ``angle`` parameter of our ``Angle`` instance. In itself, this wouldn't be very useful but given that we have a handle on our [``DynamicMap``](http://holoviews.org/reference/containers/bokeh/DynamicMap.html) ``dmap``, we can use the ``event`` method to update the ``angle`` parameter value and update the plot:

In [ ]:
dmap.event(angle=90)

*When running this cell, the visualization above will jump to the 90º position!*

This simple example shows how you can use to ``event`` method to update a visualization with any value you can generate in Python.

In [ ]:
# Exercise: Regenerate the DynamicMap, initializing the angle to 15 degrees

In [ ]:
# Exercise: Use dmap.event to set the angle shown to 145 degrees.

In [ ]:
# Exercise: Do not specify an initial angle so that the default value of 0 degrees is used.

In [ ]:
# Exercise: Use the cell magic %%output backend='matplotlib' to try the above with matplotlib

In [ ]:
# Exercise: Declare a DynamicMap using annotations2 and AngleAndSize
# Then use the event method to set the size to 1.5 and the angle to 30 degrees
def annotations2(angle, size):
    radians = (angle / 180) * np.pi
    return (hv.Box(0,0,4, orientation=np.pi/4) 
            * hv.Ellipse(0,0,(size,size*2), orientation=radians) 
            * hv.Text(0,0,'{0}º'.format(float(angle))))

AngleAndSize = Stream.define('AngleAndSize', angle=0., size=1.)